In [1]:
import subprocess
result=subprocess.run(["sudo","cpupower","frequency-set","-u","6.0GHz"],stdout=subprocess.DEVNULL,stderr=subprocess.DEVNULL)
if result.returncode==0:
    print("Success")
else:
    print("Failed")

Success


#### Breakdown of a single neural network
- X -> input
- Wx -> Weights
- bx -> bias
- A  -> Activation function
- Y -> Output

Z =W1.X +b1

Z' = A(Z)=W1.X+b1 , this node has an activation function

Y= W2.Z' + b2, this node has no activation function

- Loss functin:
- Caculate Gradient Using back propagation
- Optimizer 

### Components of pytorch
- Base class for defining custom model is `torch.nn.Module`

- Fully connected or dense layers `torch.nn.linear`

- Activation function `torch.nn.ReLU`

- Optimizer `torch.optim`

- Loss function `torch.nn.CrossEntropyLoss`

- Loads data in batch `torch.utils.data.DataLoader`


### Different way to create neural network

1. Functional:  Flexible, harder to interpret 
2. Sequential: nn.Sequential

### Building a neural network

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
## functional API

class SimpleFunctionalNN(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(SimpleFunctionalNN,self).__init__()
        self.fc1=nn.Linear(input_size,hidden_size)
        self.relu=nn.ReLU()
        self.fc2=nn.Linear(hidden_size,output_size)

    def forward(self,x):
        x=self.fc1(x)
        x=self.relu(x)
        x=self.fc2(x)
        return x

##### $$\text{Input} (x) \rightarrow \text{Linear Layer 1} \rightarrow \text{ReLU} \rightarrow \text{Linear Layer 2} \rightarrow \text{Output}$$

In [4]:
class SimpleSequentialNN(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(SimpleSequentialNN,self).__init__()
        self.network=nn.Sequential(
            nn.Linear(input_size,hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size,output_size)
        )

    def forward(self,x):
        return self.network(x)


### Training the neural network

In [5]:
model_fun1=SimpleFunctionalNN(input_size=4,hidden_size=8,output_size=3)

In [6]:
print(model_fun1)

SimpleFunctionalNN(
  (fc1): Linear(in_features=4, out_features=8, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=8, out_features=3, bias=True)
)


In [7]:
x=torch.randn(10,4) # 10 sample, 4 features
y=torch.randint(0,3,(10,))

In [8]:
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model_fun1.parameters(),lr=0.01)

In [9]:
print(x,y)

tensor([[-0.3036, -1.2135, -0.4890, -0.3544],
        [ 2.0071, -0.3807, -0.4258, -0.6304],
        [ 1.9367, -0.2227, -1.0800, -0.2277],
        [-0.3851,  0.1827, -0.9960,  0.4736],
        [-0.0516, -0.2640,  1.1861,  2.0456],
        [-0.6181, -0.2511,  1.2472, -1.4273],
        [ 0.2351, -0.1926, -0.3944,  1.0918],
        [ 0.7222, -0.5619, -0.2782,  0.2073],
        [ 0.8111, -0.6777, -1.0332,  0.8492],
        [ 0.0707, -0.8691, -0.8215, -0.2919]]) tensor([2, 1, 0, 0, 2, 0, 1, 1, 2, 2])


In [10]:
### training loop

epoch=40
for e in range(epoch):
    optimizer.zero_grad()
    outputs=model_fun1(x)
    loss=criterion(outputs,y)
    loss.backward()
    optimizer.step()
    if (e+1)%10==0:
       print(f"Epoch {e+1}/4000, Loss: {loss.item():.4f}")

Epoch 10/4000, Loss: 0.9899
Epoch 20/4000, Loss: 0.8701
Epoch 30/4000, Loss: 0.7504
Epoch 40/4000, Loss: 0.6321


In [11]:
import pandas as pd
import numpy as np

df=pd.read_csv("https://raw.githubusercontent.com/Arannamoy/datasets/refs/heads/main/Medical%20Cost%20Personal%20Datasets/insurance.csv")

df.head(5)

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [13]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
age,1338.0,39.207025,14.049960,18.0000,27.00000,39.000,51.000000,64.00000
bmi,1338.0,30.663397,6.098187,15.9600,26.29625,30.400,34.693750,53.13000
children,1338.0,1.094918,1.205493,0.0000,0.00000,1.000,2.000000,5.00000
charges,1338.0,13270.422265,12110.011237,1121.8739,4740.28715,9382.033,16639.912515,63770.42801


In [14]:
df.shape

(1338, 7)

In [15]:
df[df.duplicated()]

,age,sex,bmi,children,smoker,region,charges
581,19,male,30.59,0,no,northwest,1639.5631


In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split


In [17]:
train_df,test_df=train_test_split(df,test_size=0.25,random_state=42)


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [19]:
print(df['sex'].unique())
print(df['smoker'].unique())
print(df['region'].unique())

['female' 'male']
['yes' 'no']
['southwest' 'southeast' 'northwest' 'northeast']


In [20]:
label_encoder={}
categorical_colums=["sex","smoker","region"]
for col in categorical_colums:
    le=LabelEncoder()
    train_df[col]=le.fit_transform(train_df[col].astype(str)) ### because encoder works on str not pd series
    test_df[col]=le.transform(test_df[col].astype(str))
    label_encoder[col]=le
    print(type(test_df[col]))

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [21]:
X_train=train_df.drop(columns=['charges'])
y_train=train_df['charges']
X_test=test_df.drop(columns=['charges'])
y_test=test_df['charges']

In [22]:
print(X_train.head(5))
print(y_test.head(5))

      age  sex     bmi  children  smoker  region
693    24    1  23.655         0       0       1
1297   28    0  26.510         2       0       2
634    51    1  39.700         1       0       3
1022   47    1  36.080         1       1       2
178    46    0  28.900         2       0       3
764      9095.06825
887      5272.17580
890     29330.98315
1293     9301.89355
259     33750.29180
Name: charges, dtype: float64


In [23]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)


In [24]:
# Convert tensor
X_train_tensor=torch.tensor(X_train,dtype=torch.float32)
X_test_tensor=torch.tensor(X_test,dtype=torch.float32)
y_train_tensor=torch.tensor(y_train.values,dtype=torch.float32).view(-1,1)
y_test_tensor=torch.tensor(y_test.values,dtype=torch.float32).view(-1,1)

In [25]:
y_train_tensor.shape

torch.Size([1003, 1])

In [26]:
### define neural network model
class SimpleNNLinearRegression(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(SimpleNNLinearRegression,self).__init__()
        self.network=nn.Sequential(
            nn.Linear(input_size,hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size,output_size),
            nn.ReLU(),
            nn.Linear(output_size,1)
        )

    def forward(self,x):
        return self.network(x)

In [27]:
simple_nn_linear_regression_model=SimpleNNLinearRegression(X_train_tensor.shape[1],64,128)

In [28]:
print(simple_nn_linear_regression_model)

SimpleNNLinearRegression(
  (network): Sequential(
    (0): Linear(in_features=6, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=1, bias=True)
  )
)


In [29]:
# loss and optimizer

criterion=nn.MSELoss()
optimizer=optim.Adam(simple_nn_linear_regression_model.parameters(),lr=0.01)


In [33]:
epochs=100000
for epoch in range(epochs):
    simple_nn_linear_regression_model.train()
    optimizer.zero_grad()
    predictions=simple_nn_linear_regression_model(X_train_tensor)
    loss=criterion(predictions,y_train_tensor)
    loss.backward()
    optimizer.step()

    if(epoch+1)%10==0:
        print(f"epoch {epoch}/{epochs}, Loss: {loss.item():.4f}")



epoch 9/100000, Loss: 3998441.7500
epoch 19/100000, Loss: 3992265.7500
epoch 29/100000, Loss: 3983292.2500
epoch 39/100000, Loss: 3978816.7500
epoch 49/100000, Loss: 3978009.5000
epoch 59/100000, Loss: 3975311.5000
epoch 69/100000, Loss: 3979300.2500
epoch 79/100000, Loss: 3972655.7500
epoch 89/100000, Loss: 3967643.2500
epoch 99/100000, Loss: 3956488.5000
epoch 109/100000, Loss: 3947680.0000
epoch 119/100000, Loss: 3947783.5000
epoch 129/100000, Loss: 3943673.5000
epoch 139/100000, Loss: 3935945.0000
epoch 149/100000, Loss: 3937357.0000
epoch 159/100000, Loss: 3932633.2500
epoch 169/100000, Loss: 3939124.2500
epoch 179/100000, Loss: 3931808.7500
epoch 189/100000, Loss: 3920210.0000
epoch 199/100000, Loss: 3914233.0000
epoch 209/100000, Loss: 3910083.2500
epoch 219/100000, Loss: 3916389.5000
epoch 229/100000, Loss: 3907364.7500
epoch 239/100000, Loss: 3898714.5000
epoch 249/100000, Loss: 3907104.7500
epoch 259/100000, Loss: 3898678.0000
epoch 269/100000, Loss: 3886870.5000
epoch 279/10

In [34]:
### model evaluation
simple_nn_linear_regression_model.eval()
y_pred=simple_nn_linear_regression_model(X_test_tensor).detach().numpy()

In [ ]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score,root_mean_squared_error

In [40]:
mse=mean_squared_error(y_test_tensor.numpy(),y_pred)
mae=mean_absolute_error(y_test_tensor.numpy(),y_pred)
r2=r2_score(y_test_tensor.numpy(),y_pred)
rmse=mse**0.5
print(mse)
print(mae)
print(r2)
print(rmse)

99844560.0
6368.2119140625
0.33830076456069946
9992.224977451218


In [41]:
def predict_charges(age,sex,bmi,children,smoker,region):
    input_data=pd.DataFrame([[age,sex,bmi,children,smoker,region ]],columns=['age','sex','bmi','children','smoker','region'])
    for col in categorical_colums:
        input_data[col]=label_encoder[col].transform(input_data[col])
    input_data=scaler.transform(input_data)
    input_tensor=torch.tensor(input_data,dtype=torch.float32)
    predict_charges=simple_nn_linear_regression_model(input_tensor).item()
    return predict_charges

In [43]:
print(predict_charges(19,'female',27.9,0,"yes","southwest"))
print(predict_charges(60,'female',27.9,0,"yes","southwest"))

16928.75390625
35407.90234375


In [31]:
result=subprocess.run(["sudo","cpupower","frequency-set","-u","2.8GHz"],stdout=subprocess.DEVNULL,stderr=subprocess.DEVNULL)
if result.returncode==0:
    print("Success")
else:
    print("Failed")

Success
